# Queries

In [ ]:
data_user_welfare_surplus = '''
SELECT 
    block_number,
    `hash`,
    transaction_index as `index`,
    gas as gas_limit,
    gas_price,
    receipt_gas_used as gas_used,
    max_fee_per_gas,
    max_priority_fee_per_gas,
    MAX(gas_price-max_priority_fee_per_gas) OVER (PARTITION BY block_number) AS base_fee,
    receipt_status as status,
    transaction_type as type
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE DATE(block_timestamp) = "2023-01-01"

-- priority fee is to compute using basefee
'''

data_sim_4844 = '''
SELECT
    block_number,
    `hash`,
    transaction_index as `index`,
    gas as gas_limit,
    gas_price,
    receipt_gas_used as gas_used,
    max_fee_per_gas,
    max_priority_fee_per_gas,
    MAX(gas_price-max_priority_fee_per_gas) OVER (PARTITION BY block_number) AS base_fee,
    receipt_status as status,
    transaction_type as type,
    CASE
        WHEN (
        from_address = "0x6887246668a3b87f54deb3b94ba47a6f63f32985" -- op sequencer address
        AND to_address = "0x5e4e65926ba27467555eb562121fac00d24e9dd2" -- op CanonicalTransactionChain address
        ) THEN "op_data"
        WHEN (
        from_address = "0xc1b634853cb333d3ad8663715b08f41a3aec47cc" -- arb batch submitter address
        AND to_address = "0x1c479675ad559dc151f6ec7ed3fbf8cee79582b6" -- arb sequencer inbox
        ) THEN "arb_data"
        ELSE "other"
    END AS data_tag,
    (byte_length(input) / 2 - 1) AS data_bytes
FROM `bigquery-public-data.crypto_ethereum.transactions`
WHERE DATE(block_timestamp) = "2023-02-24"
'''

# 4844 Data for Simulation

In [2]:
from google.colab import auth
from google.cloud import bigquery

project = 'incrementai-proto'
location = 'US'
client = bigquery.Client(project=project, location=location)
auth.authenticate_user()

job = client.get_job('bquxjob_6e2a1255_186de4908d7')

df = job.to_dataframe()
df.head()

,block_number,hash,index,gas_limit,gas_price,gas_used,max_fee_per_gas,max_priority_fee_per_gas,base_fee,status,type,data_tag,data_bytes
0,16694584,0x971431be8deee130ff52febda56bdde09e3435e85d0f...,14,159222,34885439765,104935,<NA>,<NA>,32301451293,1,0,other,132.0
1,16694584,0x43c02349d35bfadb6709cf1cfb2c051a8d122056e2fa...,120,48612,32801451293,29340,45954155418,500000000,32301451293,1,2,other,68.0
2,16694584,0xb5560d0f250edd82a4abd54b8258bef625b014b8d18f...,57,124588,33301451293,119600,41403693704,1000000000,32301451293,1,2,other,36.0
3,16694584,0xcd4ba9f4413390abd616255c2774b731f8d850973880...,84,233872,32801451293,157602,45954155418,500000000,32301451293,1,2,other,1060.0
4,16694584,0x07e6da1121e833e6f9ca9a43806786711c3301b0a0d8...,67,250000,32919132298,34237,<NA>,<NA>,32301451293,1,0,other,68.0


In [3]:
import pandas as pd
import statistics as stats

## Approx max_fee and priority_fee for type 1 txns

In [7]:
bin_labels = range(1,9)
df['gas_bucket'] = pd.qcut(df['gas_used'], q=10, labels=bin_labels, duplicates='drop')

In [8]:
dfm = df[df.type==2][['block_number', 'gas_bucket', 'max_fee_per_gas', 'max_priority_fee_per_gas']].groupby(
    ['block_number', 'gas_bucket']).apply(lambda x: pd.Series(dict(
        max_fee_imputed = stats.median(x['max_fee_per_gas']),
        max_priority_fee_imputed = stats.median(x['max_priority_fee_per_gas']),
    ))).reset_index()
dfm.tail(10)

,block_number,gas_bucket,max_fee_imputed,max_priority_fee_imputed
56638,16701597,7,3.331738e+10,5.000000e+08
56639,16701597,8,3.331738e+10,5.000000e+08
56640,16701598,1,3.530946e+10,5.000000e+08
56641,16701598,2,4.011138e+10,1.250000e+09
56642,16701598,3,3.893051e+10,7.500000e+08
56643,16701598,4,3.553544e+10,5.000000e+08
56644,16701598,5,3.553544e+10,5.000000e+08
56645,16701598,6,3.553544e+10,5.000000e+08
56646,16701598,7,3.331738e+10,5.000000e+08
56647,16701598,8,3.553544e+10,5.000000e+08


In [9]:
df = df.merge(dfm, on=['block_number', 'gas_bucket'], how='inner')
df.head(10)

,block_number,hash,index,gas_limit,gas_price,gas_used,max_fee_per_gas,max_priority_fee_per_gas,base_fee,status,type,data_tag,data_bytes,gas_bucket,max_fee_imputed,max_priority_fee_imputed
0,16694584,0x971431be8deee130ff52febda56bdde09e3435e85d0f...,14,159222,34885439765,104935,<NA>,<NA>,32301451293,1,0,other,132.0,6,4.595416e+10,500000000.0
1,16694584,0xb5560d0f250edd82a4abd54b8258bef625b014b8d18f...,57,124588,33301451293,119600,41403693704,1000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
2,16694584,0xe292b6f7cef8c6abff30eba47996d4f6480191a96726...,36,116900,34301451293,110595,46000000000,2000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
3,16694584,0x41964d9d770c6a8256e88ae66cd010a386c0251f889d...,106,116900,32801451293,110595,45954155418,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
4,16694584,0x8fa3a033cd5b42cc2bf5ebba748fd60233f70ce2e60d...,47,200000,33410682710,105818,68482871585,1109231417,32301451293,1,2,other,164.0,6,4.595416e+10,500000000.0
5,16694584,0x998c815f1edd2b470629b6165cede9c55f756fc4e8ff...,113,116900,32801451293,110595,42182943234,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
6,16694584,0x5e9f1ab7d9f31b137b548f8d11b5366aed9c2e2cc48f...,132,116900,32418209449,110595,40540942256,116758156,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
7,16694584,0x81b678cf78e276a3b6a144790ef270afaed5decfb899...,104,116900,32801451293,110595,45954155418,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
8,16694584,0x3f7f7e3c5dea859d71d74d4cd5c4d5771d3f6e7200e7...,24,116900,34301451293,110595,46000000000,2000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0
9,16694584,0x3b76d38e77f2c5a29e23fb0dc770094b7070d6e68243...,77,116900,32801451293,110595,41573640362,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0


In [10]:
df['max_fee'] = df.apply(lambda x: x['max_fee_per_gas'] if x['type']==2 else max(x['max_fee_imputed'], x['gas_price']), axis=1) # to avoid negative surplus take the max of imputed max fee and price
df['max_priority_fee'] = df.apply(lambda x: x['max_priority_fee_per_gas'] if x['type']==2 else x['max_priority_fee_imputed'], axis=1)
df.head(10)

,block_number,hash,index,gas_limit,gas_price,gas_used,max_fee_per_gas,max_priority_fee_per_gas,base_fee,status,type,data_tag,data_bytes,gas_bucket,max_fee_imputed,max_priority_fee_imputed,max_fee,max_priority_fee
0,16694584,0x971431be8deee130ff52febda56bdde09e3435e85d0f...,14,159222,34885439765,104935,<NA>,<NA>,32301451293,1,0,other,132.0,6,4.595416e+10,500000000.0,4.595416e+10,5.000000e+08
1,16694584,0xb5560d0f250edd82a4abd54b8258bef625b014b8d18f...,57,124588,33301451293,119600,41403693704,1000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.140369e+10,1.000000e+09
2,16694584,0xe292b6f7cef8c6abff30eba47996d4f6480191a96726...,36,116900,34301451293,110595,46000000000,2000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.600000e+10,2.000000e+09
3,16694584,0x41964d9d770c6a8256e88ae66cd010a386c0251f889d...,106,116900,32801451293,110595,45954155418,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.595416e+10,5.000000e+08
4,16694584,0x8fa3a033cd5b42cc2bf5ebba748fd60233f70ce2e60d...,47,200000,33410682710,105818,68482871585,1109231417,32301451293,1,2,other,164.0,6,4.595416e+10,500000000.0,6.848287e+10,1.109231e+09
5,16694584,0x998c815f1edd2b470629b6165cede9c55f756fc4e8ff...,113,116900,32801451293,110595,42182943234,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.218294e+10,5.000000e+08
6,16694584,0x5e9f1ab7d9f31b137b548f8d11b5366aed9c2e2cc48f...,132,116900,32418209449,110595,40540942256,116758156,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.054094e+10,1.167582e+08
7,16694584,0x81b678cf78e276a3b6a144790ef270afaed5decfb899...,104,116900,32801451293,110595,45954155418,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.595416e+10,5.000000e+08
8,16694584,0x3f7f7e3c5dea859d71d74d4cd5c4d5771d3f6e7200e7...,24,116900,34301451293,110595,46000000000,2000000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.600000e+10,2.000000e+09
9,16694584,0x3b76d38e77f2c5a29e23fb0dc770094b7070d6e68243...,77,116900,32801451293,110595,41573640362,500000000,32301451293,1,2,other,36.0,6,4.595416e+10,500000000.0,4.157364e+10,5.000000e+08


## Get blob data

In [11]:
df['gas_used_calldata'] = df.apply(lambda x: 16*x['data_bytes'], axis=1)

In [12]:
dfo = df[['block_number', 'index', 'gas_used', 'max_fee', 'max_priority_fee', 'base_fee', 'data_tag', 'data_bytes', 'gas_used_calldata']].copy()
dfo.head()

,block_number,index,gas_used,max_fee,max_priority_fee,base_fee,data_tag,data_bytes,gas_used_calldata
0,16694584,14,104935,4.595416e+10,5.000000e+08,32301451293,other,132.0,2112.0
1,16694584,57,119600,4.140369e+10,1.000000e+09,32301451293,other,36.0,576.0
2,16694584,36,110595,4.600000e+10,2.000000e+09,32301451293,other,36.0,576.0
3,16694584,106,110595,4.595416e+10,5.000000e+08,32301451293,other,36.0,576.0
4,16694584,47,105818,6.848287e+10,1.109231e+09,32301451293,other,164.0,2624.0


In [13]:
dfo[dfo['data_tag']=='arb_data'].describe(percentiles=[0.5])

,block_number,index,gas_used,max_fee,max_priority_fee,base_fee,data_bytes,gas_used_calldata
count,1.055000e+03,1055.000000,1.055000e+03,1.055000e+03,1.055000e+03,1.055000e+03,1055.000000,1.055000e+03
mean,1.669811e+07,151.733649,1.842386e+06,5.774755e+10,9.121099e+07,3.137498e+10,99275.552607,1.588409e+06
std,1.817704e+03,57.029322,1.102684e+04,1.573698e+10,5.116870e+07,9.751770e+09,636.380071,1.018208e+04
min,1.669452e+07,6.000000,1.673783e+06,4.224785e+10,6.141737e+07,2.281213e+10,89316.000000,1.429056e+06
50%,1.669827e+07,143.000000,1.844338e+06,5.613642e+10,9.426717e+07,2.825990e+10,99396.000000,1.590336e+06
max,1.670160e+07,655.000000,1.858159e+06,1.952515e+11,1.100000e+09,9.821040e+10,100164.000000,1.602624e+06


In [14]:
dfo[dfo['data_tag']=='arb_data'].sort_values('block_number')['block_number'].drop_duplicates().diff().describe(percentiles=[0.5])

count    1029.000000
mean        6.874636
std         2.791181
min         1.000000
50%         7.000000
max        37.000000
Name: block_number, dtype: float64

In [15]:
dfo[dfo['data_tag']=='op_data'].describe(percentiles=[0.5])

,block_number,index,gas_used,max_fee,max_priority_fee,base_fee,data_bytes,gas_used_calldata
count,2.982000e+03,2982.000000,2.982000e+03,2.982000e+03,2.982000e+03,2.982000e+03,2982.000000,2.982000e+03
mean,1.669793e+07,152.003018,5.981255e+05,6.091062e+10,9.632323e+07,3.026605e+10,31071.493964,4.971439e+05
std,2.045471e+03,54.047543,3.178673e+05,1.730274e+10,5.315175e+07,8.609011e+09,19897.477949,3.183596e+05
min,1.669452e+07,9.000000,2.595720e+05,4.136784e+10,6.141737e+07,2.198410e+10,10008.000000,1.601280e+05
50%,1.669782e+07,140.000000,5.031515e+05,5.634468e+10,1.000000e+08,2.798354e+10,25185.000000,4.029600e+05
max,1.670160e+07,745.000000,2.007252e+06,2.308838e+11,1.000000e+09,1.116246e+11,119589.000000,1.913424e+06


In [16]:
dfo[dfo['data_tag']=='op_data'].sort_values('block_number')['block_number'].drop_duplicates().diff().describe(percentiles=[0.5])

count    2981.000000
mean        2.375713
std         0.778745
min         2.000000
50%         2.000000
max        12.000000
Name: block_number, dtype: float64

In [17]:
dfo.to_csv('4844-sample.csv', index=False)